In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from erisk.utils import get_spark

spark = get_spark(cores=8, memory="28g")
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/30 19:23:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/30 19:23:48 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [3]:
bucket = "gs://dsgt-clef-erisk-2024"
test_df = spark.read.parquet(f"{bucket}/task1/parquet/test")
test_df.show()

+--------+--------------------+--------------------+--------------------+---------------+--------+
|   DOCNO|                POST|                 PRE|                TEXT|_corrupt_record|filename|
+--------+--------------------+--------------------+--------------------+---------------+--------+
|   0_0_6|I'm trying to wor...|\n\nAlthough most...|I will not tell m...|           NULL|s_0.trec|
|456_1_12|You're not like '...|In general though...|Oh, and if you're...|           NULL|s_1.trec|
| 764_1_5|Maybe it's one of...|My past experienc...|But I still want ...|           NULL|s_1.trec|
|651_0_28|\n\nWe all run ba...|\n\nSo this woman...|I couldn't even i...|           NULL|s_1.trec|
| 268_1_3| Both were great,...|\n\nI've only had...|One a couple year...|           NULL|s_1.trec|
|364_0_12|I started opening...|Even though I too...|Which I can under...|           NULL|s_1.trec|
|765_0_33|Nowhere on my inv...|I ask about the t...|Words can not exp...|           NULL|s_1.trec|
|409_0_18|

In [4]:
test_df.count()

15542200

In [5]:
# lets also load up the train df
train_df = spark.read.parquet(f"{bucket}/task1/parquet/train")
train_df.show()
train_df.count()

+--------+--------------------+--------+
|   DOCNO|                TEXT|filename|
+--------+--------------------+--------+
| s_0_0_0|    1.ye katiliyorum|s_0.trec|
| s_0_1_0|ok haklsn abi gol...|s_0.trec|
| s_0_2_0| almanca yarrak gibi|s_0.trec|
| s_0_3_0|hani u oyunlarn e...|s_0.trec|
| s_0_3_1|dead cellste ygda...|s_0.trec|
| s_0_3_2|bunlarn bir dili ...|s_0.trec|
| s_0_4_0|lnce diriltiyor s...|s_0.trec|
| s_0_6_0|       ziya gzel sal|s_0.trec|
| s_0_7_0|  artk dedem deilsin|s_0.trec|
| s_0_8_0|sorma bizim matem...|s_0.trec|
| s_0_9_0|240 Volt FUCKMAST...|s_0.trec|
|s_0_10_0|bunlar nerden evi...|s_0.trec|
|s_0_11_0|beynine gidecek k...|s_0.trec|
|s_0_12_0|semeyen vizyonsuz...|s_0.trec|
|s_0_13_0|       ok haklsn abi|s_0.trec|
|s_0_14_0|ilkokul zamanlari...|s_0.trec|
|s_0_15_0|iliki kurmakta zo...|s_0.trec|
|s_0_15_1|liseye giden bir ...|s_0.trec|
|s_0_15_2|tipimin ve kiilii...|s_0.trec|
|s_0_15_3|ben insanlarla ko...|s_0.trec|
+--------+--------------------+--------+
only showing top

4264693

In [6]:
from pyspark.ml.feature import (
    Tokenizer,
    Word2Vec,
    StopWordsRemover
)
from pyspark.sql import functions as F

# let's apply tf-idf to the text column, and also include word2vec
from pyspark.ml.feature import Tokenizer, Word2Vec
from pyspark.ml import Pipeline, Transformer
from pyspark.sql.types import ArrayType, DoubleType, StringType
from sentence_transformers import SentenceTransformer
import pandas as pd

# bucket = "gs://dsgt-clef-erisk-2024"
# test_df = spark.read.parquet(f"{bucket}/task1/parquet/test")
# train_df = spark.read.parquet(f"{bucket}/task1/parquet/train")

# import sbert model
model = SentenceTransformer("all-MiniLM-L6-v2")

# # create a pandas udf that will encode the text and return an array of doubles
@F.pandas_udf(returnType=ArrayType(DoubleType()))
def encode(x: pd.Series) -> pd.Series:
    return pd.Series(model.encode(x).tolist())


word2vec_features = 64
tokenizer = Tokenizer(inputCol="TEXT", outputCol="words")

remover = StopWordsRemover(
            inputCol=tokenizer.getOutputCol(), outputCol="filtered_words"
        )

word2vec = Word2Vec(
    vectorSize=word2vec_features,
    numPartitions=8,
    inputCol=tokenizer.getOutputCol(),
    outputCol="word2vec",
)


# pipeline = Pipeline(stages=[tokenizer, remover, encode])

# pipeline_model = pipeline.fit(train_df)
# %time pipeline_model.transform(train_df.repartition(8)).show()

In [9]:
# CUSTOM TRANSFORMER ----------------------------------------------------------------
class BertTransformer(Transformer):
    """
    Wrapper for BERT to add it to the pipeline
    """

    def __init__(self, inputCol=None, outputCol=None, append_str=None):
        self.column = column
        self.model = SentenceTransformer("all-MiniLM-L6-v2")
        
    # def _transform(self, df: DataFrame) -> DataFrame:
    #     df = df.drop(*[x for x in df.columns if any(y in x for y in self.banned_list)])
    #     return df
    
    @F.pandas_udf(returnType=ArrayType(DoubleType()))
    def _transform(self, x: pd.Series) -> pd.Series:
        return pd.Series(model.encode(x).tolist())

In [23]:
from pyspark import keyword_only
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param, Params, TypeConverters
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable
from pyspark.sql import DataFrame
from pyspark.pandas import Series
from pyspark.sql.types import StringType
import pyspark.sql.functions as F
from pyspark.sql.types import ArrayType, DoubleType, StringType
from sentence_transformers import SentenceTransformer
import pandas as pd

"""
Wrapper for BERT to add it to the pipeline
"""
# CUSTOM TRANSFORMER ----------------------------------------------------------------
class BertTransformer(Transformer,               # Base class
                     HasInputCol,               # Sets up an inputCol parameter
                     HasOutputCol,              # Sets up an outputCol parameter
                     DefaultParamsReadable,     # Makes parameters readable from file
                     DefaultParamsWritable      # Makes parameters writable from file
                    ):      
    input_col = Param(Params._dummy(), "input_col", "input column name.", typeConverter=TypeConverters.toString)
    output_col = Param(Params._dummy(), "output_col", "output column name.")


    @keyword_only
    def __init__(self, input_col: str = "input", output_col: str = "output"):
        super(BertTransformer, self).__init__()
        self.model = SentenceTransformer("all-MiniLM-L6-v2")
        self._setDefault(input_col=None, output_col=None)
        kwargs = self._input_kwargs
        self.set_params(**kwargs)

    @keyword_only     
    def set_params(self, input_col: str = "input", output_col: str = "output"):
        kwargs = self._input_kwargs
        self._set(**kwargs)
    
    def get_input_col(self):
        return self.getOrDefault(self.input_col)
  
    def get_output_col(self):
        return self.getOrDefault(self.output_col)
            
    def _transform(self, df: DataFrame):

        input_col = self.get_input_col()
        output_col = self.get_output_col()
        
        @F.pandas_udf(returnType=ArrayType(DoubleType()))
        def encode(x: pd.Series) -> pd.Series:
            return pd.Series(model.encode(x).tolist())

        
        return df.withColumn(output_col, encode(input_col))
    





In [30]:

transformer = BertTransformer(input_col="TEXT", output_col="embeddings")
pipeline = Pipeline(stages=[transformer])

pipeline_model = pipeline.fit(train_df)
%time pipeline_model.transform(train_df.repartition(8)).show()

+--------------+--------------------+-----------+--------------------+
|         DOCNO|                TEXT|   filename|          embeddings|
+--------------+--------------------+-----------+--------------------+
|   s_1713_77_3|Obviously FedEx i...|s_1713.trec|[0.07817142456769...|
| s_1147_156_12|The neurologist a...|s_1147.trec|[-0.0036929193884...|
|    s_1401_9_6|On the enrollment...|s_1401.trec|[-0.0344385877251...|
|  s_1899_304_3|Why is it so sens...|s_1899.trec|[0.13363982737064...|
|   s_1473_88_1|A color code syst...|s_1473.trec|[-0.0758219882845...|
|  s_1116_171_0| Jailbait wasn't ...|s_1116.trec|[-0.0268517620861...|
|   s_1373_35_0|   u/repostsleuthbot|s_1373.trec|[-0.0888357460498...|
|   s_2077_31_2|I'm guessing ther...|s_2077.trec|[0.04538367688655...|
|  s_2137_639_0|Wegen Corona-Muta...|s_2137.trec|[-0.1068064272403...|
|  s_1511_311_0|[Source](https://...|s_1511.trec|[-0.0483171567320...|
| s_1360_403_48|They gave up the ...|s_1360.trec|[-0.0089962221682...|
|s_236

In [37]:
res_df = pipeline_model.transform(train_df)
res_df.printSchema()

root
 |-- DOCNO: string (nullable = true)
 |-- TEXT: string (nullable = true)
 |-- filename: string (nullable = true)
 |-- embeddings: array (nullable = true)
 |    |-- element: double (containsNull = true)



In [39]:
res_df.show(n=5, truncate=80)

+-------+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
|  DOCNO|                                                                            TEXT|filename|                                                                      embeddings|
+-------+--------------------------------------------------------------------------------+--------+--------------------------------------------------------------------------------+
|s_0_0_0|                                                                1.ye katiliyorum|s_0.trec|[0.0180697999894619, 0.004467027261853218, 0.020407328382134438, -4.309927753...|
|s_0_1_0|                                                     ok haklsn abi gold atar msn|s_0.trec|[-0.13028191030025482, 0.0033819167874753475, -0.027139397338032722, 0.082881...|
|s_0_2_0|                                                             almanca yarrak gibi|s_0.t

In [40]:
# generate a new dataframe with features from both train and test

total_df = (
    train_df.select(
        "DOCNO",
        "TEXT",
        "filename",
        F.lit("train").alias("dataset"),
    )
    .union(
        test_df.select(
            "DOCNO",
            F.concat(
                F.coalesce(F.col("PRE"), F.lit("")),
                F.coalesce(F.col("TEXT"), F.lit("")),
                F.coalesce(F.col("POST"), F.lit("")),
            ).alias("TEXT"),
            "filename",
            F.lit("test").alias("dataset"),
        ),
    )
    .where("filename is not null")
    .where("TEXT is not null")
).repartition(16)

In [41]:
pipeline_model = pipeline.fit(total_df)
pipeline_model.write().overwrite().save(f"{bucket}/task1/models/pipeline_sbert")
print("pipeline model saved")

res_df = pipeline_model.transform(total_df)
res_df.printSchema()

# save both the pipeline and the total_df
res_df.write.mode("overwrite").parquet(f"{bucket}/task1/parquet/combined_sbert")
print("combined sbert saved")

pipeline model saved
root
 |-- DOCNO: string (nullable = true)
 |-- TEXT: string (nullable = true)
 |-- filename: string (nullable = true)
 |-- dataset: string (nullable = false)
 |-- embeddings: array (nullable = true)
 |    |-- element: double (containsNull = true)



ERROR:root:Exception while sending command.                        (0 + 4) / 16]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java s

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 47178)
Traceback (most recent call last):
  File "/usr/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/usr/local/lib/python3.10/dist-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
  File "/usr/local/lib/python3.10/dist-packages/pyspark/accumulators.py", line 271, in accum_updates
    num_updates = read_int(

Py4JError: An error occurred while calling o306.parquet